In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/red-table/Red_Table.xlsx


In [2]:
#from google.colab import drive
#import pandas as pd
#import numpy as np
#!pip install openpyxl
#import re 
#path = "/content/sample_data/Red_Table.xlsx"

In [3]:
!pip install openpyxl
import re

     |████████████████████████████████| 242 kB 778 kB/s            


# 데이터 확인
- 데이터를 불러와서 확인후
- 3개의 번역 데이터를 통합하였습니다

In [4]:
path = "../input/red-table/Red_Table.xlsx"

In [5]:
df = pd.read_excel(path,sheet_name = '2번데이터')# 2번데이터자료

tr_1 = pd.read_excel(path,sheet_name = '번역_시도,시군구') # 번역_시도,시군구 자료
tr_2 = pd.read_excel(path,sheet_name = '번역_읍면동') #번역_읍면동 자료
tr_3 = pd.read_excel(path,sheet_name = '번역_도로명').rename(columns={'중국어 간체':'중국어_간체','중국어 번체':'중국어_번체'})
# tr_1,2와 컬럼명 통일

In [6]:
# 시도,시군구 / 읍면동 / 도로명 데이터 통합
tr = pd.concat([tr_1,tr_2,tr_3],axis=0)
tr.head()

,한국어,영어,일본어,중국어_간체,중국어_번체,시도,시군구,읍면동
0,서울특별시,Seoul,ソウル特別市,首尔特别市,首爾特別市,NaN,NaN,NaN
1,종로구,Jongno-gu,チョンノ区,钟路区,鍾路區,NaN,NaN,NaN
2,중구,Jung-gu,チュン区,中区,中區,NaN,NaN,NaN
3,용산구,Yongsan-gu,ヨンサン区,龙山区,龍山區,NaN,NaN,NaN
4,성동구,Seongdong-gu,ソンドン区,城东区,城東區,NaN,NaN,NaN


## 변환할 데이터 주소 리스트화 작업
- 일단은 공백을 기준으로 삼아서 분류했습니다.
- 또한 일관성을 위해 ',' 를 제거하고, 영어의 경우 전부 소문자로 변경하여 저장하였습니다
- 변경할 주소가 들어있는 데이터 파일에는 '시도','시군구','읍면동','로길가','번지수'라는 컬럼을 추가하였습니다

In [7]:
#tr['영어'].apply(lambda x : "".join(x.replace("-",' ').split())).to_list()[:4]

In [8]:
df[['시도','시군구','읍면동','로길가','번지수']] = ''

address_list = []
for i in df['주소'].to_list() :
    address_list.append(i.replace(","," ").lower().split())
al = address_list.copy()

# 영어 -> 한글
## 언어별로 각각 다르게 변환하기로 설정했습니다.
- 대-소문자 구분없이 전부 다 소문자로 변환하여 진행했습니다
- 또한 중복을 제거하는 것이 도움에 될것이라 판단하여 중복을 제거한 상태로 진행했습니다
- 정규표현식 : /a-zA-Z/
- 표준화를 위해 단어들 사이의 공백은 '-'로 변경하였습니다 / ex) Hyoja ro 7 gil  -> Hyoja-ro 7-gil
- {영문주소 : 한글 주소} 식으로 딕셔너리를 생성해서, 주소의 단어가 key값에 존재할 경우 한글 주소를 출력하는 식으로 진행하였습니다.

In [9]:
tr['영어'] = tr['영어'].apply(lambda x : x.lower())
eng_dict = dict(tr[['영어','한국어']].drop_duplicates().values)
# eng_dict = 영어->한국어 딕셔너리

## 변환작업 :
- 변환할 주소가 들어있는 al (address_list)
- str의 마지막 문자가 gil(길), gu(구), dong(동), ro(로)와 같이 주소를 나타내는 단어로 분류하였습니다
- 데이터가 서울시만으로 한정되어 있기때문에 도와 시 조건문은 생략하였습니다. 
- 리스트 안에 쪼개진 리스트가 들어있는 형태기때문에 for문을 2번 사용하였습니다 
- 다만 데이터에 따라 규모가 더 커질경우 반복문이 아닌 다른 방식으로 해결해야 할 것 같습니다

In [10]:
for i in al : # 100개의 주소 리스트
    for j in range(len(i)) : # 리스트 내부의 split()으로 쪼개진 str 리스트들
        if j >= 1 and i[j-1] + " " + i[j] in eng_dict : #  ['12'] ['gil'] <- 이런 방식으로 저장된 조건문의 경우
            if i[j][-3:] == "gil" :
                df['로길가'].iloc[al.index(i)]  += eng_dict[i[j-1] + " " + i[j]]
                i[j-1],i[j] = '','' # 딕셔너리에 있던 단어들은 공백으로 대체하였습니다.
        elif i[j] in eng_dict :
            if i[j][-3:] == 'gil' :
                df['로길가'].iloc[al.index(i)] += eng_dict[i[j]]
            if i[j][-2:] == 'gu' :
                df['시군구'].iloc[al.index(i)] += eng_dict[i[j]]
            elif i[j][-4:] == 'dong' :
                df['읍면동'].iloc[al.index(i)] += eng_dict[i[j]]
            elif i[j][-2:] == 'ro' :
                df['로길가'].iloc[al.index(i)] += eng_dict[i[j]]
            i[j] = ''
        elif i[j][-4:] == 'dong' :
            #print(i[j], al.index(i))
            a = i[j].replace("-", " ")
            if a[0].isdigit() :
                df['읍면동'].iloc[al.index(i)] += a[0] + "동"
                i[j] = ''
        elif 'ro' in i[j] :
            i[j] = i[j].replace('ro','ro ')
            if i[j] in eng_dict :
                df['로길가'].iloc[al.index(i)] += eng_dict[i[j]]
                i[j] = ''
            elif i[j][:i[j].index(" ")] in eng_dict :
                df['로길가'].iloc[al.index(i)] += eng_dict[i[j][:i[j].index(" ")]]
                i[j] = i[j][i[j].index(" ")+1:]

# 일본어 -> 한글
- 일본어 정규표현식 : /[ぁ-ゔ]+|[ァ-ヴー]+[々〆〤]/ 혹은    [/一-龥/+]
- 일본어도 영어와 마찬가지로 일본어 : 한국어 딕셔너리를 생성했습니다

In [11]:
jap_df = pd.concat([tr_1,tr_2,tr_3])[['일본어','한국어']]
jap_dict = dict(jap_df.values)

In [12]:
for i in al :
    for j in range(len(i)) :
        #print(i[j], re.compile('[/一-龥/+]').findall(i[j]))
        if "ソウル特別市" in i[j] : # 서울 특별시 구분
            i[j] = i[j].replace("ソウル特別市", "")
            df['시도'].iloc[i] += jpn_dict["首尔特别市"]
            
        if "市" in i[j] : # 시 구분 -> 우선 순위 x
            b = i[j][i[j].index("市"):]
            
        if "区" in i[j] : # 구 구분
            b = i[j][:i[j].index("区")+1] #  앞 글자 ~구를 의미하는 한자가 일본어 딕셔너리에 포함되어 있으면 한글 주소로 변환
            if b in jap_dict :
                al[al.index(i)][j] = al[al.index(i)][j].replace(b, "")
                al[al.index(i)].append(jap_dict_1[b])
        if i[j] in jap_dict :
            #print(i[j])
            al[al.index(i)][j] = jap_dict[i[j]]

# 중국어 -> 한글
- 중국어는 간체와 번체가 나눠져 있어 딕셔너리를 2개 생성해 하나로 통합했습니다
- 중국어도 비슷한 방식으로 진행했습니다
- 다만 간체와 번체가 똑같은 경우가 있어서 중복을 제거하였지만 효과적인 방법인지는 확실히 모른채로 진행했습니다

In [13]:
chn_df = tr[['중국어_간체','중국어_번체','한국어']].drop_duplicates()
chn_dict_1 = dict(chn_df[['중국어_간체','한국어']].values)
chn_dict_2 = dict(chn_df[['중국어_번체','한국어']].values)
chn_dict_1.update(dict(chn_df[['중국어_번체','한국어']].values))
chn_dict = chn_dict_1.copy() 

In [14]:
for i in range(len(al)) :
    for j in range(len(al[i])) :
        # 서울시 검색 -> 서울시, 서울특별시, 서울 등 을 ()으로 묶어 in으로 확인하여 코드를 간결하게 짜려했으나 수정하지 못하였습니다.
        if "首尔市" in al[i][j] :
            al[i][j] = al[i][j].replace("首尔市", "")
            df['시도'].iloc[i] += chn_dict["首尔特别市"]
        elif "首尔特别市" in al[i][j] :
            al[i][j] = al[i][j].replace("首尔特别市", "")
            df['시도'].iloc[i] += chn_dict["首尔特别市"]
        elif "首尔" in al[i][j] :
            al[i][j] = al[i][j].replace("首尔", "")
            df['시도'].iloc[i] += chn_dict["首尔特别市"]
        
        # 구 검색
        if "区" in al[i][j]: 
            b = al[i][j][:al[i][j].index("区")+1][-4:]
            for k in range(2,5) :
                if b[-k:] in chn_dict :
                    if df['시군구'].iloc[i] == "" :
                        df['시군구'].iloc[i] += chn_dict[b[-k:]]
                        al[i][j] = al[i][j].replace(b[-k:], "")
        elif "區" in al[i][j]: 
            al[i][j]
            b = al[i][j][:al[i][j].index("區")+1]
            if b in chn_dict :
                al[i][j] = al[i][j].replace(b, "")
                df['시군구'].iloc[i] += chn_dict[b]    
        
        # 로-길-가 검색
        if "街" in al[i][j] :
            b = al[i][j][:al[i][j].index("街")+1][-6:]
            for k in range(2,7) :
                if b[-k:] in chn_dict :
                    if df['로길가'].iloc[i] == "" :
                        df['로길가'].iloc[i] += chn_dict[b[-k:]]
                    al[i][j] = al[i][j].replace(b[-k:], "")                    
        # 읍면동 검색
        if "洞" in al[i][j]: 
            b = al[i][j][:al[i][j].index("洞")+1][-6:]
            for k in range(2,7) :
                if b[-k:] in chn_dict :
                    if df['읍면동'].iloc[i] == "" :
                        df['읍면동'].iloc[i] += chn_dict[b[-k:]]
                    al[i][j] = al[i][j].replace(b[-k:], "")
        # ~가 검색
        if "路" in al[i][j]: 
            b = al[i][j][:al[i][j].index("路")+1]
            for k in range(2,7) :
                if b[-k:] in chn_dict :
                    if df['읍면동'].iloc[i] == "" :
                        df['읍면동'].iloc[i] += chn_dict[b[-k:]]
                    al[i][j] = al[i][j].replace(b[-k:], "")    

In [15]:
#re.compile('[0-9-/+]+').findall(al[2][0]) #숫자와 특수문자

# 주소 리스트에 저장되어 있던 한글 주소 및 번지수

- 기존에 한글로 된 주소도 있었기 때문에 주소 리스트 재확인 (국가명은 제외하였습니다)
- 모든 주소를 한글로 변환하고 남아있는 숫자의 경우 주소의 마지막 부분이라 판단하여 번지수로 옮겼음
-

In [16]:
for i in range(len(al)) :
    for j in range(len(al[i])) :
        a = al[i][j]
        if a in ('서울시','서울특별시','서울') : 
            al[i][j] = ''
            if df['시군구'].iloc[i] == '':
                df['시도'].iloc[i] += a
        if al[i][j] in tr['한국어'].tolist() :
            if a[-1] in '구' and df['시군구'].iloc[i] == '':
                df['시군구'].iloc[i] += a
            elif a[-1] in ('읍','면','동') and df['읍면동'].iloc[i] == '':
                df['읍면동'].iloc[i] += a
            elif a[-1] in ('로','길','가') and df['로길가'].iloc[i] == '':
                df['로길가'].iloc[i] += a
            al[i][j] = ''
            

In [17]:
for i in al :
    a = re.compile('[0-9-/+]+').findall(" ".join(i))
    if len(a) == 1 :
        if df['번지수'].iloc[al.index(i)] == "" :
            df['번지수'].iloc[al.index(i)] += a[0]
    else : 
        # 숫자가 여러개 남아있는 경우 확실한 주소를 찾기 어려워 보류하였습니다,,!.
        continue

- 임시로 저장한 통합주소
- 이 주소를 가지고 도로명 주소로 변환할 생각입니다. 
- 공백도 많고 부정확한 정보도 있기에 100개 다 완벽하게 변환할 수 는 없지만,
- (영어,일본어,중국어) -> 한글주소 변환 -> 도로명 주소 변환 으로 진행할 계획이었습니다

In [18]:
df['통합주소'] = df[['시도','시군구','읍면동','로길가','번지수']].apply(lambda x : ' '.join(x.values.astype(str)), axis=1)

# 도로명 주소 변환
- 모든 주소를 한글 주소로 변환한것은 아니지만, 전부 다 변환하게 되면, api를 활용해 도로명 주소로 바꿀 예정입니다
- 카카오 우편번호 서비스 주소 검색 api 활용
- https://postcode.map.daum.net/guide#usage
- 코드 참고 : https://www.sagein.net/677

In [19]:
import requests

url = 'https://dapi.kakao.com/v2/local/search/address.json'
REST_API_KEY = 'd9a2dc86a6decd8b068022a1633d3939'
# 카카오 developers에서 API를 받아서 진행했습니다.
headers = {"Authorization": "KakaoAK {}".format(REST_API_KEY)}

def get_address(address):
    params = {'query': "{}".format(address)}
    resp = requests.get(url, params = params, headers = headers)
    if len(resp.json()['documents']) == 0 :
        return ""
    data = resp.json()['documents'][0]
    if data['road_address'] == None :
        return ""
    return data['road_address']['address_name']

In [20]:
df['제출용_주소'] = df['통합주소'].apply(lambda x: get_address(x))

In [21]:
df[['주소','제출용_주소']].to_excel('레드테이블_과제_2_김경준.xlsx', index=False)

# 리뷰
- 우선은 도로명주소 변환 작업보단, 외국어 주소를 한글주소로 변경하는게 우선인 것 같습니다.
- 적은 데이터와 엑셀을 변경하지 않는 범위 내에서 작업을 했지만, 큰 데이터를 다룰 경우 조금 더 효율적인 방식을 사용해야 할 것 같습니다.
- 문자열을 얼마나 효율적으로 다루느냐에 따라 프로젝트의 속도가 많이 달라질 것 같다고 생각했습니다.
- 또한 한자의 경우 정규표현식이나 언어 인코딩 관련하여 공부할 필요가 있다고 느꼈습니다.